<a href="https://colab.research.google.com/github/PhebeJ/Project/blob/main/IRIS_LOCALIZATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
import os
import warnings

In [ ]:
import cv2
import numpy as np

from skimage import transform, feature

In [ ]:
def read_imgs():
    """read all images of CASIA Iris Image Database (version 1.0)
    :return: the train set and test set
    :rtype: tuple (train, test)
    """

    base_dir = '/content/drive/MyDrive/CASIA Iris Image Database (version 1.0)'
    classes = os.listdir(base_dir)
    train, test = [], []
    for c in classes:
        tr_dir = '%s/%s/1/' % (base_dir, c)
        te_dir = '%s/%s/2/' % (base_dir, c)
        for f in os.listdir(tr_dir):
            if f[-3:] == 'bmp':
                train.append(cv2.imread(tr_dir + f, 0))
        for f in os.listdir(te_dir):
            if f[-3:] == 'bmp':
                test.append(cv2.imread(te_dir + f, 0))
    print(train)
    return train, test

In [ ]:
def process_imgs(imgs, use_offset=False):
    """process the input raw images to feature vectors
    :imgs: the image set
    :use_offset: in the paper, it denotes unwrapping the iris with different angles can
    remove rotation invariance. However, it seems no effect.
    :return: the train set and test set
    :rtype: tuple (train, test)
    """

    processed = []
    for img in imgs:
        circles = detect_by_hough(img)
        circles = np.array(circles).reshape(1 ,2 ,3)

        # denoising
        (_, B) = cv2.threshold(img ,180 ,255 ,cv2.THRESH_BINARY)
        (_, C) = cv2.threshold(img ,100 ,255 ,cv2.THRESH_BINARY)
        img = img & ~B & C


In [ ]:
def main():
    train_imgs, test_imgs = read_imgs()

    train = process_imgs(train_imgs, use_offset=False)
    #test = process_imgs(test_imgs, use_offset=False)

    train_labels = np.repeat(range(108), 3)
    #test_labels = np.repeat(range(108), 4)

In [ ]:
def detect_by_hough(img):
    """preprocess the image, then use hough transform to detect both pupil and iris.
    :param img: the input image
    :return: the circles of pupil and iris, (x, y, radius), (x, y, radius)
    :rtype: tuple
    """

    #I have used the roughly_localize method, since it is error-prone.

    def preprocess(img, pupil=False):
        # respectively process pupil and iris
        if pupil:
            thresh = cv2.adaptiveThreshold(img, 255,
                                           cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                           cv2.THRESH_BINARY, 5, 3)
            blur = cv2.GaussianBlur(thresh, (9, 9), 0)
            blur = cv2.medianBlur(blur, 33)
            #
            canny = cv2.Canny(blur, 15, 50)
        else:
            blured = img.copy()
            for i in range(3):
                blured = cv2.medianBlur(blured, 11)
            canny = cv2.Canny(blured, 15, 30)
            canny[:,
            pupil_circle[0] - pupil_circle[2] - 30:pupil_circle[0] + pupil_circle[2] + 30] = 0
            canny[0:pupil_circle[1] - pupil_circle[2]] = 0

        return canny

    canny = preprocess(img, pupil=True)

    pupil_circle = customed_hough_circle(canny, hough_radii=range(30, 70, 5))

    # slightly enlarge the radius of pupil
    pupil_circle[2] += 10

    canny = preprocess(img)

    iris_circle = customed_hough_circle(canny, hough_radii=range(pupil_circle[2] + 50, 150, 5))  # circles[0][0]

    # if the distance of iris center and pupil center is too far, we fix the iris center.
    if ((iris_circle - pupil_circle)[:2] ** 2).sum() ** 0.5 > pupil_circle[-1] * 0.3:
        iris_circle[:2] = pupil_circle[:2]

    return pupil_circle, iris_circle


def get_pupil_roughly(img, binarize=False):
    """roughly localize pupil, using the method introduced in the paper.
    :param img: the input image
    :param binarize: whether binarize the img firstly
    :return: the center coordinates (x, y)
    :rtype: tuple (int, int)
    """

    if binarize:
        (_, img) = cv2.threshold(img,
                                 0, 255,
                                 cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    x_min = img.sum(axis=0).argmin()
    y_min = img.sum(axis=1).argmin()

    return x_min, y_min


def select_region(img, x_min, y_min, size=60):
    """select a square  region centered at (x_min, y_min).
    :param img: the input image
    :param x_min, y_min: the center coordinates
    :param size: the range
    :return: the selected image
    :rtype: ndarray
    """

    size = min(x_min, y_min, size)
    return img[y_min - size:y_min + size, x_min - size:x_min + size]


def roughly_localize(img, size=60):
    """the whole procedure of roughly localizing pupil.
    :param img: the input image
    :param size: the range
    :return: the center coordinates (x, y)
    :rtype: tuple (int, int)
    """

    x_min, y_min = get_pupil_roughly(img)

    # the coordinates should not be off center too much.
    if abs(x_min - 160) > 30:
        x_min = 160
    if abs(y_min - 140) > 30:
        y_min = 140

    # localize 2 times.
    for i in range(2):
        x, y = get_pupil_roughly(select_region(img, x_min, y_min, size=size),
                                 binarize=True)
        if abs(x_min - 160) > 30 or abs(y_min - 140) > 30:
            break
        x_min -= size - x
        y_min -= size - y

    return x_min, y_min


def customed_hough_circle(img, hough_radii=range(30, 60)):
    """find circles on given img.
    :param img: the input image
    :param hough_radii: the radii of candidate circles
    :return: the best circle, (x, y, radius)
    :rtype: tuple (int, int, int)
    """

    hough_res = transform.hough_circle(img, hough_radii)
    centers = []
    accums = []
    radii = []

    for radius, h in zip(hough_radii, hough_res):
        num_peaks = 10
        peaks = feature.peak_local_max(h, num_peaks=num_peaks)
        centers.extend(peaks)
        accums.extend(h[peaks[:, 0], peaks[:, 1]])
        radii.extend([radius] * num_peaks)
    best = np.argsort(accums)[::-1][0]
    return np.array([centers[best][1], centers[best][0], radii[best]])

In [ ]:
if __name__ =='__main__':
    warnings.filterwarnings("ignore")
    main()

[array([[185, 185, 177, ..., 163, 165, 168],
       [192, 176, 188, ..., 171, 178, 166],
       [182, 184, 184, ..., 177, 174, 187],
       ...,
       [220, 229, 222, ..., 209, 198, 205],
       [222, 217, 222, ..., 207, 203, 200],
       [223, 224, 218, ..., 212, 209, 201]], dtype=uint8), array([[207, 219, 203, ..., 225, 222, 218],
       [219, 212, 197, ..., 221, 228, 221],
       [210, 202, 204, ..., 220, 238, 221],
       ...,
       [218, 217, 217, ..., 219, 219, 214],
       [219, 220, 220, ..., 221, 227, 210],
       [223, 216, 221, ..., 220, 218, 212]], dtype=uint8), array([[199, 203, 206, ..., 213, 212, 206],
       [206, 210, 202, ..., 223, 222, 207],
       [211, 207, 202, ..., 235, 221, 206],
       ...,
       [198, 194, 194, ..., 207, 216, 219],
       [187, 189, 194, ..., 205, 221, 219],
       [193, 189, 191, ..., 206, 220, 217]], dtype=uint8), array([[215, 227, 217, ..., 217, 205, 212],
       [221, 222, 225, ..., 213, 215, 217],
       [231, 220, 221, ..., 219, 215, 